In [0]:
import torch 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import sys

In [0]:
# class EMD(nn.Module):
#     def __init__(self, distance_matrix, classes_weights = None, cen_loss_weight= 0.7):
#         print('New EMD')
#         super(EMD, self).__init__()
#         self.distance_matrix = distance_matrix
#         self.classes_weights = classes_weights
#         self.cross_entropy = CrossEntropyLoss(weight=self.classes_weights)
#         self.cen_loss_w = cen_loss_weight
#         self.emd_loss_w = 1 - cen_loss_weight
        
#     def forward(self, outputs, labels):
        
#         # Cross entropy
#         cen_loss = self.cross_entropy(outputs, labels)
        
#         # Emd 
#         outputs = F.softmax(outputs, dim=1)
#         outputs = - torch.log2(1 - outputs + sys.float_info.epsilon)
#         dists = self.distance_matrix[labels]
#         samples_loss = (outputs * dists).sum(dim=1)
        
#         if self.classes_weights is not None:
#             samples_loss = samples_loss * self.classes_weights[labels]
#         emd_loss = samples_loss.mean()
        
#         # Final loss
#         final_loss = self.cen_loss_w * cen_loss + self.emd_loss_w * emd_loss
        
#         return final_loss

In [0]:
# class EMD2(nn.Module):
#     def __init__(self, num_classes, distance_matrix, classes_weights = None):
#         print('EMD2')
#         super(EMD2, self).__init__()
#         self.distance_matrix = distance_matrix
#         self.eye_matrix = torch.eye(num_classes).to(distance_matrix.device)
#         self.classes_weights = classes_weights
        
#     def forward(self, outputs, labels):
#         # outputs = F.softmax(outputs, dim=1)
#         true = self.eye_matrix[labels]
#         dif = outputs.cumsum(dim=1) - true.cumsum(dim=1)     
#         dif = (self.distance_matrix[labels]*dif)**2
#         loss = dif.sum(dim=1)
#         if self.classes_weights is not None:
#             loss = loss * self.classes_weights[labels]
#         return loss
#         # return loss.mean()
        

In [0]:
class EMD(nn.Module):
    def __init__(self, distance_matrix, classes_weights = None):
        print('New Cross-EMD1')
        super(EMD, self).__init__()
        self.distance_matrix = distance_matrix
        self.classes_weights = classes_weights
        
    def forward(self, outputs, labels):
        outputs = F.softmax(outputs, dim=1)
        
        rows_inds = range(labels.shape[0])
        temp = outputs[rows_inds, labels]
        aux_outputs = 1 - outputs
        aux_outputs[rows_inds, labels] = temp
        aux_outputs[aux_outputs == 0] += sys.float_info.epsilon
        samples_loss = -(torch.log2(aux_outputs) * self.distance_matrix[labels]).sum(dim=1)
        
        if self.classes_weights is not None:
            samples_loss = samples_loss * self.classes_weights[labels]
        
        # return samples_loss
        return samples_loss.mean()

In [0]:
num_classes = 5

In [0]:
# outputs = torch.FloatTensor([[0, 0, 1, 0, 0],
#                              [0, 1, 0 ,0, 0],
#                              [0, 0, 0, 1, 0]])   

# outputs = torch.FloatTensor([[0, 0, 0, 1, 0],
#                              [0, 0, 0 ,0, 1],
#                              [0, 0, 1, 0, 0]])   

# outputs = torch.FloatTensor([[0, 0, 1, 0, 0],
#                              [1, 0, 0 ,0, 0],
#                              [0, 0, 0, 0, 1]])


outputs = torch.FloatTensor([[0, 1, 0, 0, 0],
                             [1, 0, 0 ,0, 0],
                             [0, 0, 1, 0, 0]])   

labels = torch.LongTensor([1,1,1])
# labels = torch.LongTensor([2,2,2])
# labels = torch.LongTensor([3,3,3])


In [0]:
distance_matrix =  torch.FloatTensor(
                                    [[1,  7, 8, 9, 10],
                                     [7,  1, 1, 2, 3],
                                     [8,  1, 1, 1, 2],
                                     [9,  2, 1, 1, 1],
                                     [10, 3, 2, 1, 1]])

classes_weights =  torch.FloatTensor([1,1,1,1,1])

emd1 = EMD(distance_matrix, classes_weights)
emd1(outputs, labels)

New Cross-EMD


tensor([4.3280, 9.3798, 6.2863])

In [0]:
n = 5
distance_matrix = torch.torch.FloatTensor([[1, n, n, n, n],
                                           [n, 1, 1, 1, 1],
                                           [n, 1, 1, 1, 1],
                                           [n, 1, 1, 1, 1],
                                           [n, 1, 1, 1, 1]])


classes_weights =  torch.FloatTensor([1,10,1,1,1])
emd2 = EMD2(num_classes, distance_matrix, classes_weights)
emd2(outputs, labels)

NameError: ignored